In [ ]:
#@title Copyright 2023 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-community/blob/master/guides/linked/Earth_Engine_TensorFlow_DNN_from_scratch.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-community/blob/master/guides/linked/Earth_Engine_TensorFlow_DNN_from_scratch.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Introduction

This is an Earth Engine and TensorFlow demonstration notebook.  Specifically, this notebook shows:

1.   Exporting training/testing data from Earth Engine in TFRecord format.
2.   Preparing the data for use in a TensorFlow model.
2.   Training and validating a simple model in TensorFlow.
3.   Making predictions on image data exported from Earth Engine in TFRecord format.
4.   Ingesting classified image data to Earth Engine in TFRecord format.

This is intended to demonstrate a complete i/o pipeline.  For a workflow that uses a [Vertex AI Platform](https://cloud.google.com/vertex-ai) hosted model making predictions interactively, see [this example notebook](http://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/Earth_Engine_TensorFlow_Vertex_AI.ipynb).

## Imports

In [ ]:
from google.colab import auth
from pprint import pprint
import ee
import folium
import google
import json
import tensorflow as tf
from tensorflow import keras

# Define variables

This set of global variables will be used throughout.  For this demo, you must have a Cloud Storage bucket into which you can write files.

In [ ]:
# REPLACE WITH YOUR CLOUD PROJECT
PROJECT = 'your-project'

# REPLACE WITH YOUR EARTH ENGINE ASSETS PROJECT
ASSETS_PROJECT = 'your-ee-project'

# Cloud Storage bucket into which training, testing and prediction
# datasets will be written.  You must be able to write into this bucket.
# REPLACE WITH YOUR CLOUD STORAGE BUCKET
OUTPUT_BUCKET = 'your-bucket'

# Use Landsat 8 surface reflectance bands for predictors.
BANDS = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
INPUTS = BANDS + ['NDVI']

# The labels, consecutive integer indices starting from zero, are stored in
# this property, set on each point.
LABEL = 'landcover'

# These names are used to specify properties in the export of
# training/testing data and to define the mapping between names and data
# when reading into TensorFlow datasets.
FEATURE_NAMES = BANDS + [LABEL]

# Number of label values, i.e. number of classes in the classification.
N_CLASSES = 3

# Pixel scale in meters.
SCALE = 30

# File names for the training and testing datasets.  These TFRecord files
# will be exported from Earth Engine into the Cloud Storage bucket.
TRAIN_FILE_PREFIX = 'Training_demo'
TEST_FILE_PREFIX = 'Testing_demo'
file_extension = '.tfrecord.gz'
TRAIN_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TRAIN_FILE_PREFIX + file_extension
TEST_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TEST_FILE_PREFIX + file_extension

# File name for the prediction (image) dataset.  The trained model will read
# this dataset and make predictions in each pixel.
PREDICTION_INPUT_PREFIX = 'Image_pixel_demo_'

# The output path for the classified image (i.e. predictions) TFRecord file.
PREDICTION_OUTPUT_FILE = f'gs://{OUTPUT_BUCKET}/Classified_pixel_demo.TFRecord'

# Set of min-max longitudes and latitudes.  Export imagery in this region.
PREDICTION_REGION_COORDS = [-122.7, 37.3, -121.8, 38.00]

# The name of the Earth Engine asset to be created by importing
# the classified image from the TFRecord file in Cloud Storage.
PREDICTION_ASSET_ID = f'projects/{ASSETS_PROJECT}/assets/DNN_from_scratch_demo'

ATTRIBUTION = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>'

## Authentication and Initialization

In [ ]:
auth.authenticate_user()

In [ ]:
credentials, project_id = google.auth.default()
ee.Initialize(credentials, project=PROJECT, opt_url='https://earthengine-highvolume.googleapis.com')

# Get Training and Testing data from Earth Engine

To get data for a classification model of three classes (bare, vegetation, water), we need labels and the value of predictor variables for each labeled example.  We've already generated some labels in Earth Engine.  Specifically, these are visually interpreted points labeled "bare," "vegetation," or "water" for a very simple classification demo ([example script](https://code.earthengine.google.com/?scriptPath=Examples%3ADemos%2FClassification)).  For predictor variables, we'll use [Landsat 8 surface reflectance imagery](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2), bands 2-7.

## Prepare Landsat 8 imagery

First, make a cloud-masked median composite of Landsat 8 surface reflectance imagery from 2018.  Check the composite by visualizing with folium.

In [ ]:
def maskL8sr(image):
  """Cloud masking function for Landsat 8 surface reflectance."""
  qa_mask = image.select('QA_PIXEL').bitwiseAnd(31).eq(0)
  saturation_mask = image.select('QA_RADSAT').eq(0)

  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)

  return (image.addBands(optical_bands, None, True)
                .addBands(thermal_bands, None, True)
                .updateMask(qa_mask)
                .updateMask(saturation_mask).select('SR_B*.'))


# The image input data is a 2018 cloud-masked median composite.
composite = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
          .filterDate('2018-01-01', '2018-12-31').map(maskL8sr).median())

# Use folium to visualize the imagery.
mapid = composite.getMapId({'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.3})
map = folium.Map(location=[38., -122.5])

folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr=ATTRIBUTION,
    overlay=True,
    name='median composite',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

## Add pixel values of the composite to labeled points

Some training labels have already been collected for you.  Load the labeled points from an existing Earth Engine asset.  Each point in this table has a property called `landcover` that stores the label, encoded as an integer.  Here we overlay the points on  imagery to get predictor variables along with labels.

In [ ]:
# The training/testing is a dataset of points with known land cover labels.
# Sample the image at the points and add a random column.
sample = composite.sampleRegions(
  collection=ee.FeatureCollection('projects/google/demo_landcover_labels'),
  properties=[LABEL],
  scale=SCALE).randomColumn()

# Partition the sample approximately 70-30.
training = sample.filter(ee.Filter.lt('random', 0.7))
testing = sample.filter(ee.Filter.gte('random', 0.7))

# Print the first couple points to verify.
pprint({'training': training.first().getInfo()})
pprint({'testing': testing.first().getInfo()})

## Export the training and testing data

Now that there's training and testing data in Earth Engine and you've inspected a couple examples to ensure that the information you need is present, it's time to materialize the datasets in a place where the TensorFlow model has access to them.  You can do that by exporting the training and testing datasets to tables in TFRecord format ([learn more about TFRecord format](https://www.tensorflow.org/tutorials/load_data/tfrecord)) in your Cloud Storage bucket.

In [ ]:
# Make sure you can see the output bucket.  You must have write access.
print('Found Cloud Storage bucket.' if tf.io.gfile.exists('gs://' + OUTPUT_BUCKET)
    else 'Can not find output Cloud Storage bucket.')

Once you've verified the existence of the intended output bucket, run the exports.

In [ ]:
# Create the tasks.
training_task = ee.batch.Export.table.toCloudStorage(
  collection=training,
  description='Training Export',
  fileNamePrefix=TRAIN_FILE_PREFIX,
  bucket=OUTPUT_BUCKET,
  fileFormat='TFRecord',
  selectors=FEATURE_NAMES)

testing_task = ee.batch.Export.table.toCloudStorage(
  collection=testing,
  description='Testing Export',
  fileNamePrefix=TEST_FILE_PREFIX,
  bucket=OUTPUT_BUCKET,
  fileFormat='TFRecord',
  selectors=FEATURE_NAMES)

In [ ]:
# Start the tasks.
training_task.start()
testing_task.start()

### Monitor task progress

You can see all your Earth Engine tasks by listing them.  Make sure the training and testing tasks are completed before continuing.

In [ ]:
# Print all tasks.
pprint(ee.batch.Task.list())

### Check existence of the exported files

If you've seen the status of the export tasks change to `COMPLETED`, then check for the existence of the files in the output Cloud Storage bucket.

In [ ]:
print('Found training file.' if tf.io.gfile.exists(TRAIN_FILE_PATH)
    else 'No training file found.')
print('Found testing file.' if tf.io.gfile.exists(TEST_FILE_PATH)
    else 'No testing file found.')

## Export the imagery

Now that there's training and testing data, you can train a model, host the model on Vertex AI and get predictions directly in Earth Engine, as shown in [this demo](/earth-engine/guides/tf_examples#multi-class-prediction-with-a-dnn-hosted-on-vertex-ai).  Alternatively, you may want to export imagery, run prediction outside Earth Engine, then import the resultant predictions to Earth Engine as an image.  To continue with the latter approach, export whatever imagery on which you want to perform inference to the output Cloud Storage bucket as TFRecord files.

In [ ]:
# Specify patch and file dimensions.
image_export_options = {
  'patchDimensions': [256, 256],
  'maxFileSize': 104857600,
  'compressed': True
}

# Setup the task.
image_task = ee.batch.Export.image.toCloudStorage(
  image=composite,
  description='Export TFRecord imagery for inference demo',
  fileNamePrefix=PREDICTION_INPUT_PREFIX,
  bucket=OUTPUT_BUCKET,
  scale=SCALE,
  fileFormat='TFRecord',
  region=ee.Geometry.Rectangle(PREDICTION_REGION_COORDS).getInfo()['coordinates'],
  formatOptions=image_export_options,
)

In [ ]:
# Start the task.
image_task.start()

### Monitor task progress

In [ ]:
# Print all tasks.
pprint(ee.batch.Task.list())

It's also possible to monitor an individual task.  Here we poll the task until it's done.  If you do this, please put a `sleep()` in the loop to avoid making too many requests.  Note that this will block until complete (you can always halt the execution of this cell).

In [ ]:
import time

while image_task.active():
  print('Polling for task (id: {}).'.format(image_task.id))
  time.sleep(30)
print('Done with image export.')

# Data preparation and pre-processing

Read data from the TFRecord file into a `tf.data.Dataset`.  Pre-process the dataset to get it into a suitable format for input to the model.

## Read into a `tf.data.Dataset`

Here we are going to read a file in Cloud Storage into a `tf.data.Dataset`.  ([these TensorFlow docs](https://www.tensorflow.org/guide/data) explain more about reading data into a `Dataset`).



In [ ]:
# Create a dataset from the TFRecord file in Cloud Storage.
train_dataset = tf.data.TFRecordDataset(TRAIN_FILE_PATH, compression_type='GZIP')

Ensure that you can read from the file without an error by getting a raw record.  Inspect the content of the record.

In [ ]:
# Get the first record to check.
raw_record = iter(train_dataset).next()
print('raw_record:')
print(raw_record)

# Get the raw record as a numpy array.
record = raw_record.numpy()
print('record:')
print(record)

# Decode a serialized tf.Example as a proto.
example_proto = tf.train.Example.FromString(record)
print('example_proto:')
print(example_proto)

## Define the structure of your data

For parsing the exported TFRecord files, `features_dict` is a mapping between feature names (recall that `features_dict` contains the band and label names) and `float32` [`tf.io.FixedLenFeature`](https://www.tensorflow.org/api_docs/python/tf/io/FixedLenFeature) objects.  This mapping is necessary for telling TensorFlow how to read data in a TFRecord file into tensors.  Specifically, **all numeric data exported from Earth Engine is exported as `float32`**.

(Note: *features* in the TensorFlow context (i.e. [`tf.train.Feature`](https://www.tensorflow.org/api_docs/python/tf/train/Feature)) are not to be confused with Earth Engine features (i.e. [`ee.Feature`](https://developers.google.com/earth-engine/apidocs/ee-feature)), where the former is a protocol message type for serialized data input to the model and the latter is a geometry-based geographic data structure.)

In [ ]:
# List of fixed-length features, all of which are float32.
columns = [
  tf.io.FixedLenFeature(shape=(), dtype=tf.float32) for k in FEATURE_NAMES
]

# Dictionary with names as keys, features as values.
features_dict = dict(zip(FEATURE_NAMES, columns))

pprint(features_dict)

## Parse the dataset

Now we need to make a parsing function for the data in the TFRecord files.  The data comes in flattened 2D arrays per record and we want to use the first part of the array for input to the model and the last element of the array as the class label.  The parsing function reads data from a serialized [`Example` proto](https://www.tensorflow.org/api_docs/python/tf/train/Example) into a dictionary in which the keys are the feature names and the values are the tensors storing the value of the features for that example.

In [ ]:
def parse_tfrecord(example_proto):
  """The parsing function.

  Read a serialized example into the structure defined by featuresDict.

  Args:
    example_proto: a serialized Example.

  Returns:
    A tuple of the predictors dictionary and the label, cast to an `int32`.
  """
  parsed_features = tf.io.parse_example(example_proto, features_dict)
  labels = parsed_features.pop(LABEL)
  return parsed_features, tf.cast(labels, tf.int32)

# Map the function over the dataset.
parsed_dataset = train_dataset.map(parse_tfrecord, num_parallel_calls=5)

# Print the first parsed record to check.
pprint(iter(parsed_dataset).next())

Note that each record of the parsed dataset contains a tuple.  The first element of the tuple is a dictionary with bands for keys and the numeric value of the bands for values.  The second element of the tuple is a class label.

## Create additional features

Another thing we might want to do as part of the input process is to create new features, for example NDVI, a vegetation index computed from reflectance in two spectral bands.  Here are some helper functions for that.

In [ ]:
def normalized_difference(a, b):
  """Compute normalized difference of two inputs.

  Compute (a - b) / (a + b).

  Args:
    a: an input tensor with shape=[1]
    b: an input tensor with shape=[1]

  Returns:
    The normalized difference as a tensor.
  """
  return tf.math.divide_no_nan((a - b), (a + b))

def add_NDVI(features):
  """Add NDVI to the dataset.
  Args:
    features: a dictionary of input tensors keyed by feature name.
    label: the target label

  Returns:
    A tuple of the input dictionary with an NDVI tensor added and the label.
  """
  features['NDVI'] = normalized_difference(features['SR_B5'], features['SR_B4'])
  return features

# Model setup

The basic workflow for classification in TensorFlow is:

1.  Create the model.
2.  Train the model (i.e. `fit()`).
3.  Use the trained model for inference (i.e. `predict()`).

Here we'll create a neural network model using Keras.  Note that the model used here is purely for demonstration purposes and hasn't gone through any performance tuning.

## Create the Keras model

Before we create the model, there's still a bit of pre-processing to get the data into the right input shape and a format that can be used with cross-entropy loss.  Specifically, Keras expects a list of inputs and a one-hot vector for the class. (See [the Keras loss function docs](https://keras.io/losses/), [the TensorFlow categorical identity docs](https://www.tensorflow.org/guide/feature_columns#categorical_identity_column) and [the `tf.one_hot` docs](https://www.tensorflow.org/api_docs/python/tf/one_hot) for details).

Here we will use a simple neural network model with a 64 node hidden layer, a dropout layer and an output layer.  Use a custom pre-processing `Layer` to add an NDVI input feature and stack the inputs into a vector.  The following defines the pre-processing layer, wraps the model in the preprocessing layer, compiles it, and fits it to the training data.

In [ ]:
# The base model takes a vector of inputs and returns a vector of outputs.
inputs = keras.Input(shape=(len(INPUTS)), name='input_array')
x = tf.keras.layers.Dense(64, activation=tf.nn.relu)(inputs)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(N_CLASSES, activation=tf.nn.softmax)(x)
model = tf.keras.Model(inputs=inputs, outputs=x)

# A Layer to add an NDVI feature and stack the input tensors.
class MyPreprocessing(keras.layers.Layer):
  def __init__(self, **kwargs):
    super(MyPreprocessing, self).__init__(**kwargs)

  def call(self, features_dict):
    features_dict = add_NDVI(features_dict)
    return tf.stack([features_dict[b] for b in INPUTS], axis=1)

# A Model that wraps the base model with the preprocessing layer.
class MyModel(keras.Model):
  def __init__(self, preprocessing, backbone, **kwargs):
    super().__init__(**kwargs)
    self.preprocessing = preprocessing
    self.backbone = backbone

  def call(self, features_dict):
    x = self.preprocessing(features_dict)
    return self.backbone(x)

wrapped_model = MyModel(MyPreprocessing(), model)

In [ ]:
# Compile the model with the specified loss function.
wrapped_model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Fit the model to the training data.
wrapped_model.fit(x=parsed_dataset.batch(4), epochs=5)

## Check model accuracy on the test set

Now that we have a trained model, we can evaluate it using the test dataset.  To do that, read and prepare the test dataset in the same way as the training dataset.  Here we specify a batch size of 1 so that each example in the test set is used exactly once to compute model accuracy.

In [ ]:
test_dataset = (
  tf.data.TFRecordDataset(TEST_FILE_PATH, compression_type='GZIP')
    .map(parse_tfrecord, num_parallel_calls=5)
    .batch(1))

wrapped_model.evaluate(test_dataset)

# Use the trained model to classify an image from Earth Engine

Now it's time to classify the image that was exported from Earth Engine.  If the exported image is large, it will be split into multiple TFRecord files in its destination folder.  There will also be a JSON sidecar file called "the mixer" that describes the format and georeferencing of the image.  Here we will find the image files and the mixer file, getting some info out of the mixer that will be useful during model inference.

## Find the image files and mixer file in Cloud Storage

Use `gsutil` to locate the files of interest in the output Cloud Storage bucket.  Check to make sure your image export task finished before running the following.

In [ ]:
# Get a list of all the files in the output bucket.
files_list = !gsutil ls 'gs://'{OUTPUT_BUCKET}
# Get only the files generated by the image export.
exported_files_list = [s for s in files_list if PREDICTION_INPUT_PREFIX in s]

# Get the list of image files and the JSON mixer file.
image_files_list = []
mixer_file = None
for f in exported_files_list:
  if f.endswith('.tfrecord.gz'):
    image_files_list.append(f)
  elif f.endswith('.json'):
    mixer_file = f

# Make sure the files are in the right order.
image_files_list.sort()

pprint(image_files_list)
print(mixer_file)

## Read the mixer file

The mixer contains metadata and georeferencing information for the exported patches, each of which is in a different file.  Read the mixer to get some information needed for prediction.

In [ ]:
# Load the contents of the mixer file to a JSON object.
mixer_text = !gsutil cat {mixer_file}
# Get a single string w/ newlines from the IPython.utils.text.SList
mixer = json.loads(mixer_text.nlstr)
pprint(mixer)

## Read the images on which to perform prediction into a dataset

You can feed the list of files (`imageFilesList`) directly to the `TFRecordDataset` constructor to make a combined dataset on which to perform inference.  The input needs to be preprocessed differently than the training and testing.  Mainly, this is because the pixels are written into records as patches, we need to read the patches in as one big tensor (one patch for each band), then flatten them into lots of little tensors.

In [ ]:
# Get relevant info from the JSON mixer file.
patch_width = mixer['patchDimensions'][0]
patch_height = mixer['patchDimensions'][1]
patches = mixer['totalPatches']
patch_dimensions_flat = [patch_width * patch_height, 1]

# Note that the tensors are in the shape of a patch, one patch for each band.
image_columns = [
  tf.io.FixedLenFeature(shape=patch_dimensions_flat, dtype=tf.float32)
    for k in BANDS
]

# Parsing dictionary.
image_features_dict = dict(zip(BANDS, image_columns))

# Note that you can make one dataset from many files by specifying a list.
image_dataset = tf.data.TFRecordDataset(image_files_list, compression_type='GZIP')

# Inspect a single record.
record = iter(image_dataset).next().numpy()
ex = tf.train.Example.FromString(record)
print(list(ex.features.feature.keys()))
print(len(ex.features.feature['SR_B1'].float_list.value))

In [ ]:
# Parsing function.
def parse_image(example_proto):
  return tf.io.parse_example(example_proto, image_features_dict)

# Parse the data into tensors, one long tensor per patch.
prediction_dataset = image_dataset.map(parse_image, num_parallel_calls=5)

## Generate predictions for the image pixels

To get predictions in each pixel, run the image dataset through the trained model using `model.predict()`.  Print the first prediction to see that the output is a list of the three class probabilities for each pixel.  Running all predictions might take a while.

In [ ]:
# Run prediction in batches, with as many steps as there are patches.
predictions = wrapped_model.predict(prediction_dataset, verbose=1)

# Note that the predictions come as a numpy array.  Check the first one.
print(predictions.shape)
print(predictions[0])

## Write the predictions to a TFRecord file

Now that there's a list of class probabilities in `predictions`, it's time to write them back into a file, optionally including a class label which is simply the index of the maximum probability.  We'll write directly from TensorFlow to a file in the output Cloud Storage bucket.

Iterate over the list, compute class label and write the class and the probabilities in patches.  Specifically, we need to write the pixels into the file as patches in the same order they came out.  The records are written as serialized `tf.train.Example` protos.  This might take a while.

In [ ]:
print('Writing to file ' + PREDICTION_OUTPUT_FILE)

In [ ]:
def get_empty_example():
  return tf.train.Example(
      features=tf.train.Features(
        feature={
          'label': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=[])),
          'bareProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=[])),
          'vegProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=[])),
          'waterProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=[])),
        }
      )
    )

# Instantiate the writer.
writer = tf.io.TFRecordWriter(PREDICTION_OUTPUT_FILE)

# Every patch-worth of predictions we'll dump an example into the output
# file with a single feature that holds our predictions. Since our predictions
# are already in the order of the exported data, the patches we create here
# will also be in the right order.
patch = 1
pred = 0
e = get_empty_example()
for prediction in predictions:
  e.features.feature['label'].float_list.value.extend([tf.argmax(prediction, -1)])
  e.features.feature['bareProb'].float_list.value.extend([prediction[0]])
  e.features.feature['vegProb'].float_list.value.extend([prediction[1]])
  e.features.feature['waterProb'].float_list.value.extend([prediction[2]])
  pred += 1
  if (pred == patch_width * patch_height):
    print('Done with patch ' + str(patch) + ' of ' + str(patches) + '...')
    writer.write(e.SerializeToString())
    patch += 1
    pred = 0
    e = get_empty_example()

writer.close()

# Upload the classifications to an Earth Engine asset

## Verify the existence of the predictions file

At this stage, there should be a predictions TFRecord file sitting in the output Cloud Storage bucket.  Use the `gsutil` command to verify that the predictions image (and associated mixer JSON) exist and have non-zero size.

In [ ]:
!gsutil ls -l {PREDICTION_OUTPUT_FILE}

## Upload the classified image to Earth Engine

Upload the image to Earth Engine directly from the Cloud Storage bucket with the [`earthengine` command](https://developers.google.com/earth-engine/command_line#upload).  Provide both the image TFRecord file and the JSON file as arguments to `earthengine upload`.

In [ ]:
print('Uploading to ' + PREDICTION_ASSET_ID)

In [ ]:
# Of course, you also need to authenticate to the command line.
!earthengine authenticate --auth_mode=notebook

In [ ]:
# Start the upload.
!earthengine upload image --asset_id={PREDICTION_ASSET_ID} --pyramiding_policy=mode {PREDICTION_OUTPUT_FILE} {mixer_file}

## Check the status of the asset ingestion

You can also use the Earth Engine API to check the status of your asset upload.  It might take a while.  The upload of the image is an asset ingestion task.

In [ ]:
ee.batch.Task.list()

## View the ingested asset

Display the vector of class probabilities as an RGB image with colors corresponding to the probability of bare, vegetation, water in a pixel.  Also display the winning class using the same color palette.

In [ ]:
predictions_image = ee.Image(PREDICTION_ASSET_ID)

prediction_vis = {
  'bands': 'label',
  'min': 0,
  'max': 2,
  'palette': ['red', 'green', 'blue']
}
probability_vis = {'bands': ['bareProb', 'vegProb', 'waterProb'], 'max': 0.5}

prediction_map_id = predictions_image.getMapId(prediction_vis)
probability_map_id = predictions_image.getMapId(probability_vis)

map = folium.Map(location=[37.6413, -122.2582])
folium.TileLayer(
  tiles=prediction_map_id['tile_fetcher'].url_format,
  attr=ATTRIBUTION,
  overlay=True,
  name='prediction',
).add_to(map)
folium.TileLayer(
  tiles=probability_map_id['tile_fetcher'].url_format,
  attr=ATTRIBUTION,
  overlay=True,
  name='probability',
).add_to(map)
map.add_child(folium.LayerControl())
map